In [12]:
import sys
import h5py
import numpy as np
import healpy as hp
import tensorflow as tf
import random as python_random
import nnhealpix.layers
import matplotlib.pyplot as plt
from tensorflow.keras import metrics
import keras
import pandas as pd
import os, shutil
from loss_functions import sigma_loss, sigma_f_loss, sigma2_loss,sigma_batch_loss,sigma_norm_loss,sigma_log_loss,mse_tau,mse_sigma

In [13]:
#metrics= [sigma_loss,sigma2_loss,sigma_batch_loss,sigma_norm_loss,sigma_log_loss,mse_tau,mse_sigma]
metrics= [sigma_loss,sigma_batch_loss,mse_tau,mse_sigma,sigma_f_loss]
home_dir="/home/amorelli/r_estimate/B_maps_white_noise/results_16_5_23/"
r_like=[0.0,0.001,0.005,0.007,0.01] #these are the r_test of the maps on which i test the NN 
sigma_exact=[0.000225922*2, 0.000398311, 0.001017338,0.001142098,0.0015324*2] #these are the sigma computed for the 
#r_test using the likelihood approach
load_dir='/home/amorelli/r_estimate/B_maps_white_noise/' 
to_load=['test_data_r0000_t006_150.npz','test_data_r0001_t006_15.npz','test_data_r0005_t006_241.npz',
         'test_data_r0007_t006_241.npz','test_data_r0010_t006_24.npz'] #these are the test maps
loss_kind="sigma" #var
loss_training=sigma_batch_loss 
test_model_folder="test_model"

In [14]:
try:
    hyperparameters=pd.read_csv(home_dir+"output.txt",delim_whitespace=True, index_col=None) #i try to read the 
    #file with the hyperparameters used in the training
except:
    print("no file to read") 
    dataframe={}
    dataframe["no data"]=["-"]
    hyperparameters=pd.DataFrame(dataframe)#if there is no file to read (because i analyse an old model) i create an 
    #empty dataframe
finally:
    None #i use this so that the program continue after the try,except
print(hyperparameters)

      name             loss  noise  p_stopping  p_reduce  f_reduce  \
0  16_5_23  new_sigma_batch      4          20         5       0.5   

              stop-reduce      lr  batch_size  n_layers  nodes_layers comments  
0  val_mse_batch-val_loss  0.0003          32      True            48           


In [15]:
files=[]
for name in to_load: 
    files.append(np.load(load_dir+name) )
x_test=[]
y_test=[]
for i,f in enumerate(files):#i take the maps and the corresponding r_test for each input file
    x_test.append(f["x_test"])
    y=np.zeros(len(x_test[i]))+f["y_test"] #y_test is a single number in the file -> i need to transform it in an array
    #of the same length of x_test
    y_test.append(y.reshape(y.shape[0],1))

In [16]:
model = keras.models.load_model(
    home_dir+test_model_folder,  custom_objects={'loss_training' : loss_training, 'metrics' : metrics}, 
    compile=False
) #i restore the model from the test_model folder. I need to specify the custom objects and recompile the model with the custom
#objects, thus the metrics and the loss functions
model.compile(loss=loss_training,optimizer=tf.optimizers.Adam(), metrics=metrics)

results=[]
predictions=[]
for x,y in zip(x_test,y_test): #i compute the predictions of the model for each test_set and save them in a dict
    results.append(model.evaluate(x,y,verbose=2) )
    predictions.append(model.predict(x))
npz_dict={}
for i,p in enumerate(predictions):
    npz_dict[f"sigma_{r_like[i]}"]=p[:,1]
np.savez(home_dir+"sigma_out", **npz_dict)

313/313 - 7s - loss: 1.8331e-06 - sigma_loss: 4.0980e-06 - sigma_batch_loss: 1.8313e-06 - mse_tau: 1.7760e-06 - mse_sigma: 2.3220e-06 - sigma_f_loss: 232.2026 - 7s/epoch - 22ms/step
313/313 [==============================] - 7s 20ms/step
313/313 - 6s - loss: 5.5308e-06 - sigma_loss: 1.1123e-05 - sigma_batch_loss: 5.5259e-06 - mse_tau: 5.2251e-06 - mse_sigma: 5.8978e-06 - sigma_f_loss: 589.7819 - 6s/epoch - 19ms/step
313/313 [==============================] - 6s 20ms/step
313/313 - 6s - loss: 4.9085e-05 - sigma_loss: 8.1006e-05 - sigma_batch_loss: 4.9058e-05 - mse_tau: 4.7225e-05 - mse_sigma: 3.3782e-05 - sigma_f_loss: 3378.1519 - 6s/epoch - 19ms/step
313/313 [==============================] - 6s 19ms/step
313/313 - 6s - loss: 3.7026e-05 - sigma_loss: 6.8996e-05 - sigma_batch_loss: 3.7010e-05 - mse_tau: 3.6281e-05 - mse_sigma: 3.2715e-05 - sigma_f_loss: 3271.5105 - 6s/epoch - 19ms/step
313/313 [==============================] - 6s 20ms/step
313/313 - 6s - loss: 4.9930e-05 - sigma_loss: 

In [17]:
print(npz_dict[f"sigma_{r_like[0]}"])

[0.00026497 0.00026497 0.00026497 ... 0.00026497 0.00026497 0.00026497]


In [ ]:
if loss_kind=="sigma": #if the output of the NN is directly the std i do some operations and print some values, if
    #it gives the variance i do in a different way
    r_pred=[]
    sigma_pred=[]
    r_est=[]
    sigma_r=[]
    sigma_est=[]
    sigma_sigma=[]
    mae_r=[]
    skeweness_r=[]
    skeweness_sigma=[]
    median_r=[]
    median_sigma=[]
    for i,p in enumerate(predictions): #i compute mean, std, etc for each set of predictions and save them in the corresponding
        #dictionaries (see above)
        r_pred.append(p[:,0])
        sigma_pred.append(p[:,1])
        r_est.append(np.mean(r_pred[i]))
        sigma_r.append(np.std(r_pred[i]))
        sigma_est.append(np.mean(sigma_pred[i]))
        sigma_sigma.append(np.std(sigma_pred[i]))
        mae_r.append(np.sum(np.abs(r_pred[i]-r_est[i]))/len(r_pred[i]))
        skeweness_r.append(np.sum(((r_pred[i]-r_est[i])/sigma_r[i])**3)/len(r_pred[i]))
        skeweness_sigma.append(np.sum(((sigma_pred[i]-sigma_est[i])/sigma_sigma[i])**3)/len(r_pred[i]))
        median_r.append(np.median(r_pred[i]))
        median_sigma.append(np.median(sigma_pred[i]))
    #r_test=np.asarray([int(x*100)/100 for x in r_est])
    #sigma_like=np.empty_like(r_test,dtype=float)
    #for r in r_test:
        #i=r_like.index(r)
        #sigma_like[i]=sigma_exact[i]
    d={}# i create a dictionary that stores the value of the following keywords for each test_set
    d["r_test"]=r_like
    d["r_est"]=r_est
    d["dr"]=np.abs(np.asarray(r_like)-np.asarray(r_est))/r_est
    d["median_r-r_est"]=np.asarray(median_r)-np.asarray(r_est)
    d["mae_r"]=mae_r
    d["skeweness_r"]=skeweness_r
    d["sigma_r"]=sigma_r
    d["sigma_est"]=sigma_est
    d["dsigma"]=np.abs(np.asarray(sigma_r)-np.asarray(sigma_est))/sigma_est
    d["dlike"]=np.abs(np.asarray(sigma_r)-np.asarray(sigma_exact))/sigma_r
    d["sigma_sigma"]=sigma_sigma
    d["median_sigma-sigma_est"]=np.asarray(median_sigma)-np.asarray(sigma_est)
    d["skeweness_sigma"]=skeweness_sigma

    r_data=pd.DataFrame(d)

    #define new DataFrame as original DataFrame with each row repeated 3 times. I do this because i want to join the hyperpar. dataframe
    #with the r_data dataframe and the hyperparam. dataframe only has one row
    hp_new = pd.DataFrame(np.repeat(hyperparameters.values, len(r_est), axis=0))
    #assign column names of original DataFrame to new DataFrame
    hp_new.columns = hyperparameters.columns

    out_df = pd.concat([hp_new, r_data], axis=1) # i join the two dataframes so that i can copy-paste all the info for one model directly
    #in excel

    print(out_df)
    out_df.to_csv(home_dir+'results.txt', index=False, sep=';')

In [ ]:
if loss_kind=="sigma": 
    for i in range(len(r_pred)):
        print("on_test_set r =",r_like[i])
        print("r_est:",r_est[i],"  This is the average of the $r_{i}^{NN}$ estimated by the network on the test set.")
        print("sigma_r:",sigma_r[i], " This is the std of the $r_i^{NN}$ array (std of the prediction).")
        print("sigma_r_est:", sigma_est[i], "  This is the average of the $\sigma_{i}$ estimated by the network on the test set.")
        print("\n")

In [ ]:
if loss_kind=="sigma":
    fig, ax = plt.subplots(len(r_pred),2, figsize = (12,20))
    for i in range(len(r_pred)): #for each test_set i create an histogram of the r_pred and sigma_pred and print both 
        bin_size=100
        bin_edges= np.histogram_bin_edges(r_pred[i], bins='fd')
        print("n_of_bins:",len(bin_edges))
        counts_tau, bins_tau = np.histogram(r_pred[i], bins=bin_edges) 
        bin_edges= np.histogram_bin_edges(sigma_pred[i], bins='fd')
        counts_sigma, bins_sigma = np.histogram(sigma_pred[i], bins=bin_edges)
        plt.subplot(len(r_pred),2,2*i+1)
        plt.stairs(counts_tau, bins_tau, label="r",  color='k')
        plt.axvline(r_like[i], color="black", linestyle="--", label="r_test")
        #plt.axvline(median_r[i], color="black", linestyle="--", label="r_test")
        plt.axvline(r_est[i], color="red", linestyle="-", label="r_est")
        plt.legend()
        plt.xlabel("$r$")
        plt.ylabel("$counts$")
        plt.ticklabel_format(style='sci',useOffset=True, axis='x', scilimits=[-1, 1])
        plt.subplot(len(r_pred),2,2*i+2)
        plt.stairs(counts_sigma, bins_sigma, label="std estimated by NN",  color='k')
        plt.axvline(sigma_r[i], color="black", linestyle="--", label="std on the r estimated by NN")
        plt.axvline(sigma_est[i], color="red", linestyle="-", label="Average of the std estimated by NN")
        plt.legend()
        plt.ticklabel_format(style='sci',useOffset=True, axis='x', scilimits=[-1, 1])
        plt.xlabel("$\sigma_{r}^{NN}$")
        plt.ylabel("$counts$")
        plt.legend()
        #print(counts_tau, bins_tau)
        #print(np.sort(r_pred[i]))
    #plt.savefig('results_on_test.jpg') #must be in same cell or result image will be blank

In [ ]:
if loss_kind!="sigma":
    r_pred=[]
    var_pred=[]
    r_est=[]
    var_r=[]
    var_est=[]
    sigma_var=[]
    mae_r=[]
    skeweness_r=[]
    skeweness_var=[]
    median_r=[]
    median_var=[]
    for i,p in enumerate(predictions):
        r_pred.append(p[:,0])
        var_pred.append(p[:,1])
        r_est.append(np.mean(r_pred[i]))
        var_r.append(np.std(r_pred[i])**2)
        var_est.append(np.mean(var_pred[i]))
        sigma_var.append(np.std(var_pred[i]))
        mae_r.append(np.sum(np.abs(r_pred[i]-r_est[i]))/len(r_pred[i]))
        skeweness_r.append(np.sum(((r_pred[i]-r_est[i])/sigma_r[i])**3)/len(r_pred[i]))
        skeweness_var.append(np.sum(((var_pred[i]-var_est[i])/sigma_var[i])**3)/len(r_pred[i]))
        median_r.append(np.median(r_pred[i]))
        median_var.append(np.median(var_pred[i]))
    r_test=np.asarray([int(x*100) for x in r_est])
    var_like=np.empty_like(r_test)
    for r in r_test:
        i=r_like.index(r)
        var_like[i]=sigma_exact[i]**2
    d["r_test"]=r_test
    d["r_est"]=r_est
    d["dr"]=np.abs(r_test-r_est)/r_est
    d["median_r-r_est"]=median_r-r_est
    d["mae_r"]=mae_r
    d["skeweness_r"]=skeweness_r
    sigma_r=var_r**{0.5}
    d["sigma_r"]=sigma_r
    sigma_est=var_est**{0.5}
    d["sigma_est"]=sigma_est
    d["dsigma"]=np.abs(sigma_r-sigma_est)/sigma_est
    d["dlike"]=np.abs(sigma_r-sigma_like)/sigma_r
    d["sigma_var"]=sigma_var
    d["median_var-var_est"]=median_var-var_est
    d["skeweness_var"]=skeweness_var

    r_data=pd.DataFrame(d)

    #define new DataFrame as original DataFrame with each row repeated 3 times
    hp_new = pd.DataFrame(np.repeat(hyperparameters.values, len(r_est), axis=0))

    #assign column names of original DataFrame to new DataFrame
    hp_new.columns = hyperparameters.columns

    out_df = pd.concat([hp_new, r_data], axis=1)

    print(out_df)
    out_df.to_csv(base_dir+'results.txt', index=False, sep=' ')

In [ ]:
if loss_kind!="sigma":
    for i in range(len(r_pred)):
        print("r_est:",r_est[i],"  This is the average of the $r_{i}^{NN}$ estimated by the network on the test set.")
        print("var_r:",sigma_r[i], " This is the variance of the $r_i^{NN}$ array (variance of the prediction).", "\t The standard deviation is thus: ", np.sqrt(sigma_tau[i]))
        print("var_r_est:", sigma_r[i], "  This is the average of the $\sigma_{i}$ estimated by the network on the test set.", "\t thus taking the squareroot:",np.sqrt(sigma_est[i]), "We call it like that because $\sigma_{i}$ should be an estimate of the variance in this case → its squareroot should be an estimate of the standard deviation")
        print("\n")

In [ ]:
if loss_kind!="sigma":
    fig, ax = plt.subplots(len(r_pred),2, figsize = (10,10))
    for i in range(len(r_pred)):
        counts_tau, bins_tau = np.histogram(r_pred[i])
        counts_sigma, bins_sigma = np.histogram(sigma_pred[i])
        plt.subplot(len(r_pred),2,2*i+1)
        ax[2*i+1].stairs(counts_tau, bins_tau, label="r",  color='k')
        plt.axvline(r_test[i], color="black", linestyle="--", label="r_test")
        plt.axvline(r_est[i], color="red", linestyle="-", label="r_est")
        plt.legend()
        plt.xlabel("$r$")
        plt.ylabel("$counts$")
        plt.subplot(len(r_pred),2,2*i+2)
        ax[2*i+2].stairs(counts_sigma, bins_sigma, label="std estimated by NN",  color='k')
        plt.axvline(sigma_r[i], color="black", linestyle="--", label="std on the r estimated by NN")
        plt.axvline(sigma_est[i], color="red", linestyle="-", label="Average of the std estimated by NN")
        plt.legend()
        plt.xlabel("$\sigma_{r}^{NN}$")
        plt.ylabel("$counts$")
        plt.legend()